In [ ]:

import os
import sys
import subprocess
import gc
import numpy as np
import pandas as pd
import cv2
import keras
import tensorflow as tf

import math
import heapq
from functools import partial, reduce
import functools
import random
from kaggle_datasets import KaggleDatasets
from tqdm.notebook import tqdm as tqdm

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!cp -r /kaggle/input/model-package/* /kaggle/working

In [ ]:
from classification_models.tfkeras import Classifiers

SEResNext50, preprocess_input = Classifiers.get('seresnext50')

In [ ]:
tf.keras.backend.clear_session()
HEIGHT = 137
WIDTH = 236
SIZE = 128
BATCH_SIZE = 512
GLOBAL_PIXEL_STATS = (np.array([0.06922849]),np.array([0.20515700]))

In [ ]:
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=SIZE, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    img = cv2.resize(img,(size,size))
    
    mean, std = GLOBAL_PIXEL_STATS
    img = (img.astype(np.float32)- mean) / std
    return cv2.resize(img,(size,size))

In [ ]:
if tf.test.is_gpu_available(cuda_only=True):
    config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
    sess = tf.compat.v1.Session(config=config) 
    tf.compat.v1.keras.backend.set_session(sess)


In [ ]:

# pred_resnet_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape = (128,128,1), classes=168)
# pred_model = tf.keras.layers.Dropout(rate=0.3)(pred_resnet_model.output)
# pred_model = tf.keras.layers.Flatten()(pred_model)
# pred_model = tf.keras.layers.Dense(1024, activation = "relu")(pred_model)
# pred_model = tf.keras.layers.Dropout(rate=0.3)(pred_model)
# pred_dense = tf.keras.layers.Dense(512, activation = "relu")(pred_model)

# pred_head_root = tf.keras.layers.Dense(168, activation = 'softmax', name='grapheme_out')(pred_dense)
# pred_head_vowel = tf.keras.layers.Dense(11, activation = 'softmax', name='vowel_out')(pred_dense)
# pred_head_consonant = tf.keras.layers.Dense(7, activation = 'softmax', name='consonant_out')(pred_dense)

# pred_model = tf.keras.Model(inputs=pred_resnet_model.input, outputs=[pred_head_root, pred_head_vowel, pred_head_consonant])

# pred_model.compile(
#         optimizer=tf.keras.optimizers.Adam(lr=1e-4),
#         loss={
#               #"fc1000": categorical_focal_loss(gamma=2.0, alpha=0.25), #"categorical_crossentropy",
#               "grapheme_out":  "sparse_categorical_crossentropy",
#             "vowel_out":  "sparse_categorical_crossentropy",
#             "consonant_out":  "sparse_categorical_crossentropy",
#         },
#         metrics=['top_k_categorical_accuracy', 'accuracy'])

# pred_model.load_weights('/kaggle/input/bangla-keras-models/weights.10.h5')

In [ ]:
seresnext_model = SEResNext50(input_shape=(128,128,1), include_top=False)
intermediate_layer = tf.keras.layers.GlobalAveragePooling2D()(seresnext_model.layers[-1].output)
intermediate_layer = tf.keras.layers.BatchNormalization()(intermediate_layer)#(seresnext_model.layers[-3].output) #GLOBAL AVG POOLED LAYER
intermediate_layer = tf.keras.layers.Dropout(0.5)(intermediate_layer)

head_root = tf.keras.layers.Dense(168, activation = 'softmax', name='grapheme')(intermediate_layer)
head_vowel = tf.keras.layers.Dense(11, activation = 'softmax', name='vowel')(intermediate_layer)
head_consonant = tf.keras.layers.Dense(7, activation = 'softmax', name='consonant')(intermediate_layer)

pred_model = tf.keras.Model(inputs=[seresnext_model.input], outputs=[head_root, head_vowel, head_consonant])

pred_model.compile(
  optimizer=tf.keras.optimizers.Adam(lr=1e-4),
  loss={
      "grapheme":  "sparse_categorical_crossentropy",
      "vowel":  "sparse_categorical_crossentropy",
      "consonant":  "sparse_categorical_crossentropy",
  },
  loss_weights={
      'grapheme': 0.5,
      'vowel': 0.25,
      'consonant': 0.25
  },
  metrics=['sparse_top_k_categorical_accuracy', 'accuracy'])

pred_model.load_weights('/kaggle/input/bangla-keras-models/SeResNext50_Balanced_Mix_TPU.h5')

In [ ]:
pred_files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.startswith('test_image_data'):
            pred_files.append(os.path.join(dirname, filename))

In [ ]:
def test_batch_generator(df, batch_size):
    num_imgs = len(df)

    for batch_start in range(0, num_imgs, batch_size):
        curr_batch_size = min(num_imgs, batch_start + batch_size) - batch_start
        idx = np.arange(batch_start, batch_start + curr_batch_size)

        names_batch = df.iloc[idx, 0].values
        imgs_batch = 255 - df.iloc[idx, 1:].values.reshape(-1, HEIGHT, WIDTH).astype(np.uint8)
        X_batch = np.zeros((curr_batch_size, SIZE, SIZE, 1))
        
        for j in range(curr_batch_size):
            img = (imgs_batch[j,]*(255.0/imgs_batch[j,].max())).astype(np.uint8)
            img = crop_resize(img, size=SIZE)
            img = img[:, :, np.newaxis]
            X_batch[j,] = img

        yield X_batch, names_batch

In [ ]:
row_id = []
target = []
# iterative over the test sets
for fname in pred_files:
    test_ = pd.read_parquet(fname)
    test_gen = test_batch_generator(test_, batch_size=BATCH_SIZE)

    for batch_x, batch_name in test_gen:
        # prediction
        batch_predict = pred_model.predict(batch_x)
        for idx, name in enumerate(batch_name):
            row_id += [
                f"{name}_grapheme_root",
                f"{name}_vowel_diacritic",
                f"{name}_consonant_diacritic"
                
            ]
            target += [
                np.argmax(batch_predict[0], axis=1)[idx],
                np.argmax(batch_predict[1], axis=1)[idx],
                np.argmax(batch_predict[2], axis=1)[idx],
            ]

    del test_
    gc.collect()
    
    
df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

df_sample.to_csv('submission.csv',index=False)
gc.collect()

In [ ]:
!mkdir junk
!mv ./* ./junk
!mv ./junk/submission.csv ./
!rm -rf junk